In [68]:
# Import necessary modules
import numpy as np
import pandas as pd
from math import exp

In [167]:
# Initialize Network
def initialize_network(inputs, n_hiddens, n_hidden_neurons, n_outputs):
    # Create network with 2 hidden layers
    network = {}
    network['input_layer'] = {}
        
    n = 1
    while(n <= n_hiddens):
        network['hidden_' + str(n) + '_layer'] = {}
        if n == 1:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                    = np.random.random((n_hidden_neurons, len(inputs)+1))
        else:
            network['hidden_' + str(n) + '_layer']['weights']\
                                                = np.random.random((n_hidden_neurons, n_hidden_neurons+1))
        n += 1
    
    network['output_layer']= {} 
    network['output_layer']['weights'] = np.random.random((n_outputs, n_hidden_neurons+1))
    
    return network

In [168]:
# Calculate neuron
def calculate_neuron(weights, inputs):
    inputs_plus_bias = np.append(inputs, 1)
    outputs = np.dot(weights, inputs_plus_bias)
    
    return outputs

In [169]:
# Activate neuron using Sigmoid 
def activate_sigmoid(output):
    return 1.0 / (1.0 + output)

In [170]:
# Forward propagate inputs
def forward_propagate(network, inputs):
    for layer, values in network.items():
        if layer == 'input_layer':
            continue
        print(values['weights'])
        
        
        outputs = calculate_neuron(values['weights'], inputs)
        values['outputs'] = outputs
        print(values['outputs'])
        print()
        inputs = values['outputs']
    return inputs                 

In [171]:
np.random.seed(2023)
inputs = np.array([14.88, 14.57, 0.8811, 5.554])
network = initialize_network(inputs, 3, 2, 3)

for key, value in network.items():
    print(key, value)

input_layer {}
hidden_1_layer {'weights': array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
       [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]])}
hidden_2_layer {'weights': array([[0.45637326, 0.50138226, 0.39446855],
       [0.1511723 , 0.36087518, 0.16207701]])}
hidden_3_layer {'weights': array([[0.33795869, 0.18032328, 0.3909914 ],
       [0.03564821, 0.56486165, 0.20346149]])}
output_layer {'weights': array([[0.32060446, 0.37656378, 0.18405414],
       [0.10395184, 0.45492722, 0.19586384],
       [0.37852542, 0.93053196, 0.76015971]])}


In [146]:
np.dot(np.array([[0.33795869, 0.18032328],[0.3909914 , 0.03564821],[0.56486165, 0.20346149]]), 
       np.array([9.0109182 , 8.60431288]))

array([4.59687603, 3.82991987, 6.84056844])

In [73]:
a = np.append(inputs, 1)
b = np.array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
              [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]])
np.dot(b, a)

array([19.12722975, 11.382317  ])

In [174]:
output = forward_propagate(network, inputs)
print()
print()
print(output)

[[0.3219883  0.89042245 0.58805226 0.12659609 0.14134122]
 [0.46789559 0.02208966 0.72727471 0.52438734 0.54493524]]
[19.12722985 11.38231707]

[[0.45637326 0.50138226 0.39446855]
 [0.1511723  0.36087518 0.16207701]]
[14.83051671  7.16118009]

[[0.33795869 0.18032328 0.3909914 ]
 [0.03564821 0.56486165 0.20346149]]
[6.69442093 4.77721886]

[[0.32060446 0.37656378 0.18405414]
 [0.10395184 0.45492722 0.19586384]
 [0.37852542 0.93053196 0.76015971]]
[4.12924294 3.06504811 7.73952306]



[4.12924294 3.06504811 7.73952306]


In [175]:
for layer, values in network.items():
    print(layer, values)

input_layer {}
hidden_1_layer {'weights': array([[0.3219883 , 0.89042245, 0.58805226, 0.12659609, 0.14134122],
       [0.46789559, 0.02208966, 0.72727471, 0.52438734, 0.54493524]]), 'outputs': array([19.12722985, 11.38231707])}
hidden_2_layer {'weights': array([[0.45637326, 0.50138226, 0.39446855],
       [0.1511723 , 0.36087518, 0.16207701]]), 'outputs': array([14.83051671,  7.16118009])}
hidden_3_layer {'weights': array([[0.33795869, 0.18032328, 0.3909914 ],
       [0.03564821, 0.56486165, 0.20346149]]), 'outputs': array([6.69442093, 4.77721886])}
output_layer {'weights': array([[0.32060446, 0.37656378, 0.18405414],
       [0.10395184, 0.45492722, 0.19586384],
       [0.37852542, 0.93053196, 0.76015971]]), 'outputs': array([4.12924294, 3.06504811, 7.73952306])}
